#Loading Data ( Data Ingestion) :-
As LLMS can't direclty read from a PDF file, webpage natively  , 
LangChain or any other framework loads these and convert it into a Document object containing Text and MetaData .

In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("BTPMaterials.pdf")
documents=loader.load()

In [13]:
documents[:5]

[Document(page_content='IS 1893  (Part 1) : 2025  \n \n                                   \n \n           \nडिज़ाइन भूकंपीय जोडिम  और संरचऩाओं \nके भूकम्परोधी डिज़ाइन के म़ानदंि — \nरीडि संडिि़ा \n भ़ाग 1 स़ाम़ान्य प्ऱावध़ान  \n ( सातवाां पुनरीक्षण ) \n \n \nDesign Earthquake Hazard  \nand Criteria for Earthquake -Resistant \nDesign of Structures — Code of Practice  \nPart 1 General Provisions   \n ( Seventh  Revision )  \n \nICS 91.120.25  \n \n  \n \n\uf0d3 BIS 2025  \n \n \n \nभारतीय  मानक ब्यूरो \nBUREAU OF INDIAN STANDARDS  \nमानक भवन, 9 बहादुर शाह ज़फर मार्ग, नई ददल्ली 110002  \nMANAK BHA VAN, 9 BAHADUR S HAH ZAFAR MARG  \nNEW DELHI 110002  \nwww.bis.gov.in      www.standardsbis.in  \n \nJune  2025                                                      Price Group X  \n \nभारतीय  मानक  \nIndian Standard  \n ', metadata={'source': 'BTPMaterials.pdf', 'page': 0}),
 Document(page_content=' \n \n Earthquake Engineering Se ctional Committee, CED 39  \n \n \n \n \nFOREWORD  \n \nThis In

# Transform  - LangChain Text Splitter is used.
This step splits the long text into smaller, manageable segments called "chunks."
So that this don't exceed the Context Window 

#Chunk Overlap 
Think of it like a sliding window. Instead of cutting the text cleanly at character 1000 and starting the next chunk at 1001, you might cut at 1000 but start the next chunk at 800. The text between 800 and 1000 appears in both chunks.

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) 
finalDocuments=text_splitter.split_documents(documents)
finalDocuments

[Document(page_content='IS 1893  (Part 1) : 2025  \n \n                                   \n \n           \nडिज़ाइन भूकंपीय जोडिम  और संरचऩाओं \nके भूकम्परोधी डिज़ाइन के म़ानदंि — \nरीडि संडिि़ा \n भ़ाग 1 स़ाम़ान्य प्ऱावध़ान  \n ( सातवाां पुनरीक्षण ) \n \n \nDesign Earthquake Hazard  \nand Criteria for Earthquake -Resistant \nDesign of Structures — Code of Practice  \nPart 1 General Provisions   \n ( Seventh  Revision )  \n \nICS 91.120.25  \n \n  \n \n\uf0d3 BIS 2025  \n \n \n \nभारतीय  मानक ब्यूरो \nBUREAU OF INDIAN STANDARDS  \nमानक भवन, 9 बहादुर शाह ज़फर मार्ग, नई ददल्ली 110002  \nMANAK BHA VAN, 9 BAHADUR S HAH ZAFAR MARG  \nNEW DELHI 110002  \nwww.bis.gov.in      www.standardsbis.in  \n \nJune  2025                                                      Price Group X  \n \nभारतीय  मानक  \nIndian Standard', metadata={'source': 'BTPMaterials.pdf', 'page': 0}),
 Document(page_content='Earthquake Engineering Se ctional Committee, CED 39  \n \n \n \n \nFOREWORD  \n \nThis Indian Standar

##Vector Embeddings 
-Its a technique where we converts text into Vectors
- For creating a VectorDB we need embeddings

In [17]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(finalDocuments,OpenAIEmbeddings())


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


## Query - 
We basiclly do search from DB

In [22]:
query="What are the different earthquake zones where India is divided?"

result=db.similarity_search(query)
result[:2]

[Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New Delhi.  \n7  Town falling at the boundary of zones demarcation line between two zones shall be considered in higher zone.  \n \nFIG. 1 MAP OF INDIA SHOWING EARTHQUAKE ZONES', metadata={'page': 22, 'source': 'BTPMaterials.pdf'}),
 Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New Delhi.  \n7  Town falling at the boundary of zones demarcation line between two zones shall be considered in higher zone.  \n \nFIG. 1 MAP OF INDIA SHOWING EARTHQUAKE ZONES', metadata={'page': 22, 'source': 'BTPMaterials.pdf'})]

In [28]:
# Convert query to embedding first
embeddings=OpenAIEmbeddings()
query_embedding=embeddings.embed_query("  What are the different earthquake zones where India is divided?") 
result_by_vector=db.similarity_search_by_vector(query_embedding)
result_by_vector

[Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New Delhi.  \n7  Town falling at the boundary of zones demarcation line between two zones shall be considered in higher zone.  \n \nFIG. 1 MAP OF INDIA SHOWING EARTHQUAKE ZONES', metadata={'page': 22, 'source': 'BTPMaterials.pdf'}),
 Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New Delhi.  \n7  Town falling at the boundary of zones demarcation line between two zones shall be considered in higher zone.  \n \nFIG. 1 MAP OF INDIA SHOWING EARTHQUAKE ZONES', metadata={'page': 22, 'source': 'BTPMaterials.pdf'}),
 Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New Delhi.  \n7  Town falling at the boundary of zones demarcation line between two zones shall be considered in higher zone.  \n \nFIG. 1 MAP OF INDIA SHOWING EARTHQUAKE ZONES', metadata={'page': 22, 'source': 'BTPMaterials.pdf'}),
 Document(page_content='for Seismology (NCS), Ministry of Earth Sciences, New D